# Ejecercio 1

## Introduccion
---

Se desarrollo un ChatBot con las siguientes caracteristicas :
* El chatbot se espcializa en vulnerabilidades informaticas, nutriendose de todos los CVE reportados por la fundacion [Mitre](https://www.cve.org/Downloads).
* Todos los CVE reportados fueron tokenizados y los embeddings fueron persistidos en la siguiente [base de datos vectorial](https://drive.google.com/file/d/1ZT0fkKeYiiImxB0HHfFnsYjYDy2wYd8Z/view?usp=sharing) con motor <p>ChromaDB</p>.
* Utiliza la base de datos de grafos que provee [WikiData](https://query.wikidata.org/) para consultar mas informacion sobre los CVE encontrados. Se realizan queries en SparQL para dicho fin mediante consultas ejecutadas hacia la API de WikiData.
* El chatbot utiliza distintos modelos para diferentes tareas:
 - Para deteccion de lenguage [papluca/xlm-roberta-base-language-detection](https://huggingface.co/papluca/xlm-roberta-base-language-detection)
 - Para traduccion de texto [facebook/nllb-200-distilled-600M](https://huggingface.co/facebook/nllb-200-distilled-600M)
 - Para generacion de texto [HuggingFaceH4/zephyr-7b-beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta)
 - Deteccion NER en la horaciones [vblagoje/bert-english-uncased-finetuned-pos](https://huggingface.co/vblagoje/bert-english-uncased-finetuned-pos)
* El Chabot soporta deteccion y traduccion automatica de lenguage para ingles, frances y espanol.
* Tambien se puede utilizar el markdown ```<_ID_>CVE ID DE EJEMPLO</_ID_/>``` por si se desea consultar por un CVE especifico.





## Caracteristicas y Clases Principales
---

###  ChromaManager
---
Las clases fueron disenadas para ser adaptadas facilmente a otros temas y otras arquitecturas de trabajo:
- La calse ChromaManager soporta conexiones locales y remota a base de datos con motor Chroma. Por defecto intenta conectar a <b>localhost:8000</b> autenticandose con <p>local:local</p>.
- Se creo un Dockerfile para poder correr chroma en entornos locales para testeo.
- En colab utiliza el filesystem con sqlite para conectarse a la base de datos ya generada con todos los CVE cargados.
- Los CVE fueron cargados utilizando los CVE-IDS como metadatos y sus respectivas descripciones tokenizadas extraidas del CSV descargado de la fundacion Mitre.

### BaseDataset
---
Esta clase fue pensanda para ser flexible y reautilazada facilmente con varias caractiristicas utiles, como cargar datasets desde CSV, generar tokens automaticamente, generar pipelines y encodes automaticamente y poder persistir esta informacion en disco.

Esta clase fue disenada para ser heredada por otras clases de tipo Dataset.

Se utiliza esta clase para realizar el finetuning de modelos.

### NER
---
Checkpoint: ```vblagoje/bert-english-uncased-finetuned-pos```

La clase NER hereda de <b> BaseDataset </b>.  
Esta clase carga datos desde un CSV, detecta los principales componentes de la oracion, filtrando por los metadatos ```["VERB", "NOUN", "PROPN", "ADJ" ]```  y con un score superior por defecto al ```0.7```

### Translator
---
Checkpoint: ```facebook/nllb-200-distilled-600M```

Esta clase hereda de <b> BaseDataset </b> cuya funcion principal es traducir los entradas de texto recibidas.


### WikiData
---

Esta clase es la que realiza cosultas SparQL hacia WikiData, utilizando como <b>ITEM</b> principal los CVE IDs obtenidos al procesar los datos de entradas del usuario sobre los distintos modelos de datos y base de datos chroma.

### Trainer
---
Esta clase fue generada para entrenar y generar finetunings de distintos modelos, se muestra un ejemplo mas adelante con GPT2.
Los recursos de hardware no eran suficientes para generar un modelo los suficientemente bueno para la bot, pero se dejo para estudio academico.













### ChatBot
---
Checkpoints:
- ```papluca/xlm-roberta-base-language-detection```
- ```HuggingFaceH4/zephyr-7b-beta```

Esta clase es corazon del proyecto. Tiene el siguiente flujo de trabajo :

- Recibe un input de texto generado por el usuario
- Verifica si hay algun etiqueta en el texto de entrada ```<_ID_> UN ID </_ID_/>```
- Detecta el lenguaje.
- Si no es un idioma soportado, devuelve un mensaje por defecto.
- Traduce al ingles
- Tokeniza la entrada de texto y busca en chroma posibles coincidencias
- Busca aumentar el volumen de datos buscando informacion en WikiData con los CVE obtenidos de chroma.
- Busca en la fundacion mitre para obtener mas datos sobre las vulnerabilidades de interes.
- Genera texto de respuesta con todos los datos obtenidos. El modelo ``` https://huggingface.co/HuggingFaceH4/zephyr-7b-beta ``` es pesado por lo que se programo la clase para poder ejecutar tanto local o remoto generando llamdas a la API de Hugging Face. El modelo acepta el siguiente formato de inputs para generar texto :
```
<|system|>
Contexto que necesita el chat para responder de forma mas precisa</s>
<|user|>
Entrada de texto del usuario ? </s>
<|assistant|>
```

- Se rellena los datos de ```<system>``` con texto de todo lo procesado anteriormente. Ejemplo:

```
<|system|>

    You are Tuia a security expert who always responds using context in the style of technician.

        CVE-2017-8238: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists in a camera function.
                CVE-2016-10342: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists in a syscall handler.
                CVE-2015-9029: In all Android releases from CAF using the Linux kernel, a vulnerability exists in the access control settings of modem memory.
                CVE-2017-8236: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists in an IPA driver.
                CVE-2014-9962: In all Android releases from CAF using the Linux kernel, a vulnerability exists in the parsing of a DRM provisioning command.   
                CVE-2017-8241: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists in a WLAN function due to an incorrect message length.
                CVE-2017-8237: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists while loading a firmware image.
                CVE-2015-9025: In all Android releases from CAF using the Linux kernel, a buffer overflow vulnerability exists in a QTEE application.
                CVE-2016-10340: In all Android releases from CAF using the Linux kernel, an integer underflow leading to buffer overflow vulnerability exists in a syscall handler.
                CVE-2022-31681: VMware ESXi contains a null-pointer deference vulnerability. A malicious actor with privileges within the VMX process only, may create a denial of service condition on the host.
                CVE-2017-8238: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists in a camera function.
                CVE-2016-10342: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists in a syscall handler.
                CVE-2015-9029: in all android releases from caf using the linux kernel a vulnerability exists in the access control settings of modem memory.  
                CVE-2017-8236: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists in an ipa driver.
                CVE-2014-9962: in all android releases from caf using the linux kernel a vulnerability exists in the parsing of a drm provisioning command.    
                CVE-2017-8241: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists in a wlan function due to an incorrect message length.
                CVE-2017-8237: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists while loading a firmware image.  
                CVE-2015-9025: in all android releases from caf using the linux kernel a buffer overflow vulnerability exists in a qtee application.
                CVE-2016-10340: in all android releases from caf using the linux kernel an integer underflow leading to buffer overflow vulnerability exists in a syscall handler.
                CVE-2022-31681: vmware esxi contains a null-pointer deference vulnerability. a malicious actor with privileges within the vmx process only may create a denial of service condition on the host.
                </s>

                    <|user|>
                    Generate a list of vulnerabilities in the Linux kernel</s>

            <|assistant|>
```

- Se puede ejecutar el script chat.py y para verificar y se obtienen resultados como el siguiente :

```
Idiom: es
Question: generar una lista de vulnerabilidades en kernel de linux
Answer: 1. CVE-2021-31569: Existe una vulnerabilidad en el controlador de procesador Cortex-M cortex_m en el núcleo Linux que permite a un atacante local evacuar los recursos del sistema mediante la construcción de un dispositivo malicioso que e. 2. CVE-2022-0729: Una condición de carrera en el controlador ARM Trusted Execution Environment ARM TEE en el núcleo Linux puede ser explotada por un usuario malicioso local para obtener acceso privilegiado. 3. CVE-2022- 23176: Una vulnerabilidad de sobre flujo de buffer basada en la pila sale de la pila TCP/IP de la adelfia para Linux
-------------------------------------------------
```

```
-------------------------------------------------
Idiom: fr
Question: générer une liste de vulnérabilités dans le noyau linux
Answer: 1. CVE-2021-20716: Un problème a été découvert dans le système de fichiers EXT4 dans Linux 5.10 et antérieurs. Un attaquant local avec certains privilèges peut contourner certains permissions de fichier et écraser des fichiers arbitraires CVE-2021-20801: Une vulnérabilité de corruption de la mémoire existe dans le pilote de système de fichiers XFS dans les versions du noyau Linux pr Un attaquant local, avec certains privilèges, peut exploiter cette vulnérabilité et potentiellement exécuter un code arbitraire, entraînant un déni de service système
-------------------------------------------------
```

```
-------------------------------------------------
Idiom: en
Question: make a list of linux kernel vulnerabilities
Answer: Note: This list is not exhaustive and only includes known and reported Linux kernel vulnerabilities. Always ensure that your operating system and kernel are up to date with the latest security patch S. 1. CVE-2021-38581: A use-after-free vulnerability exists in the network filter Connection Tracking subsystem of the Linux kernel. A local attacker with privileged access can exploit this vulnerability  to bypass certain security restrictions and execute arbitrary code. 2. CVE-2021-42511: An elevation of privilege vulnerability exists in the Linux Security Module LSM implementation for SCAP securi ty target in kernel versions
-------------------------------------------------
```
```
-------------------------------------------------
Idiom: es
Question: necesito que me des informacion sobre esta vulnerabilidad cve-2017-11302
Answer: Claro, basado en el ID CVE proporcionado, CVE-2017-11302, esta es una vulnerabilidad encontrada en Adobe InDesign, una aplicación de publicación de escritorio. Esto significa que un usuario malicioso podría explotar esta vulnerabilidad para ejecutar código no autorizado y malicioso en un sistema afectado.  La versión específica que se ve afectada es InDesign 12.1.0 y versiones anteriores. Adobe ha lanzado una actualización de seguridad para abordar esta vulnerabilidad, por lo que le recomendaría actualizar su software a la la versión de ensayo para mitigar el riesgo.
-------------------------------------------------
```
```
You: Podrias darme informacion acerca de vulnerabilidades relacionadas con NFS
Idiom Detected: es
Question Translated: podrias darme informacion acerca de vulnerabilidades relacionadas con nfs
Answer: Claro, aquí hay alguna información sobre vulnerabilidades relacionadas con NFS: 1. CVE-2017-100213: Esta vulnerabilidad, también conocida como "NFS idmap bug", afecta a las implementaciones de NFSv4 que manejan el remaping de ID del usuario.  La explotación exitosa permite a un atacante eludir el contexto de seguridad de un host Linux y exportar el acceso raíz. El cliente NFS. Podría permitir que los atacantes remotos eviten ciertos obstáculos en el acceso a archivos, incluidos los destinados a restringir el acceso a un sistema de archivos solo para lectura
-------------------------------------------------
You: Puedes darme datos de este CVE-123123-Invenado
Idiom Detected: es
Question Translated: puedes darme datos de este cve-123123-invenado
Answer: Lo siento, pero no hay CVE-123123 en el contexto dado. por favor proporcione el identificador CVE correcto para que pueda ayudarle más. una vez que tenga el identificador CVE correcto, puedo recoger t - Él necesita información para usted.
-------------------------------------------------
You: exit
```



## Ejecucion
---


### Clonando Repositorio

In [31]:
! git clone https://github.com/amaranto/chatbot || ( cd chatbot/ && git pull origin main)

fatal: destination path 'chatbot' already exists and is not an empty directory.
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 4), reused 7 (delta 4), pack-reused 0
Unpacking objects: 100% (7/7), 174.24 KiB | 2.03 MiB/s, done.
From https://github.com/amaranto/chatbot
 * branch            main       -> FETCH_HEAD
   51f6280..786d32e  main       -> origin/main
Updating 51f6280..786d32e
Fast-forward
 src/chat.py            |   2 +-
 src/config.py          |   2 +-
 src/images/diagram.png | Bin 0 -> 222949 bytes
 3 files changed, 2 insertions(+), 2 deletions(-)
 create mode 100644 src/images/diagram.png


### Descarga de base de datos ChromaDB con CVE cargados

In [1]:
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

data_zip = '1ZT0fkKeYiiImxB0HHfFnsYjYDy2wYd8Z'

download = drive.CreateFile({'id': data_zip})
download.GetContentFile('data.zip')

In [9]:
! test -d data || unzip data.zip


In [12]:
! test -d chatbot/src/data/  || mv data/ chatbot/src/

### Instalar Requerimientos

In [28]:
! pip install chromadb sentence_transformers SPARQLWrapper accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00


In [15]:
! pip install -U TensorFlow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

###  Hugging Face Token

Puede usar Hugging Face para ejecutar el modelo generador de texto configurando la variable de entorno HFACE_TOKEN environment.
Si se omite ejecutara el modelo en el GPU si esta disponible o en su defecto CPU.

In [ ]:
! export HFACE_TOKEN='YOUR TOKEN' && export ENV='colab'

### Ejemplo de como usar ChromaManager
Se utilizara la base de datos chromas descargada en pasos anteriores.


Por defecto ChromaManager tratara de conectar a <b>localhost:8000</b> autenticandose con la siguientes credenciales <b>local:local</b> .

gen_vector_db.py es un script simple que carga un dataset y lo carga en la base de datos de chroma.

```python
from libs.Chroma import ChromaManager
from config import ENV, CHROMADB_PATH, CSV_SOURCE, CSV_NER_PATH

logger = logging.getLogger(__name__)

# Set to False to generate the database
# Set to True to skip the generation and use the existing database
SKIP_CSV_TO_CHROMA = True

db = ChromaManager(use_local_fs=ENV=="colab", path=CHROMADB_PATH)
db.create_collection("cve")

ds = NerDataset()

batch = 1000
total_batches = 200
total_records = batch * total_batches
add_header = not os.path.isfile(CSV_NER_PATH)
mapper = {"feature": lambda x: cleaning(x, max_length=512).lower()}

if not SKIP_CSV_TO_CHROMA:
    for i in range(0, total_records, batch):
        tokenized_data_batches = ds.load_csv_data(path=CSV_SOURCE, feature="Description",  truncate_before=i, truncate_after=i+batch, label="Name", mappers=mapper )
        ds.pipeline("ner", args={"grouped_entities": True})
        ds.generate_enteties(extra_meta_labels=["cve", "vulnerability"])
        doc = ds.to_chroma_documents()
        tokenized_data_batches.insert_document("cve", doc)
        ds.scores_to_csv(CSV_NER_PATH, header=add_header)
        add_header = False

'''
Example of search
'''
print("Example of search")
r = db.search(collection_name="cve", query="gain root access in linux kernel", k=10)

print(r.keys())
print(r["ids"])
print(r["distances"])
print(r["metadatas"])
print(r["documents"])
```

In [20]:
! export ENV='colab' && cd chatbot/src && python3 gen_vector_db.py

Se truncaron las últimas líneas 5000 del resultado de transmisión.
dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])
[['CVE-1999-0018', 'CVE-1999-0317', 'CVE-1999-0055', 'CVE-1999-0072', 'CVE-2022-33267', 'CVE-1999-0293', 'CVE-2017-5123', 'CVE-2020-12962', 'CVE-1999-0082', 'CVE-1999-0123']]
[[0.06235969066619873, 0.06303524971008301, 0.06709986925125122, 0.06908988952636719, 0.07108783721923828, 0.07110714912414551, 0.07356637716293335, 0.07454532384872437, 0.07610762119293213, 0.07834100723266602]]
[[{'NOUN': 'root privileges', 'VERB': 'allows'}, {'ADJ': 'local', 'NOUN': 'users', 'PROPN': 'linux', 'VERB': 'gives'}, {'NOUN': 'root access', 'PROPN': 'sun', 'VERB': 'allow'}, {'ADJ': 'local', 'NOUN': 'users', 'PROPN': 'aix xdat', 'VERB': 'gives'}, {'NOUN': 'drm request', 'PROPN': 'linux', 'VERB': 'sending'}, {'NOUN': 'authorization', 'PROPN': 'cisco systems', 'VERB': 'execute'}, {'ADJ': 'insufficient', 'NOUN': 'sandboxes', 'PROPN': 'linux', 'VERB': 'e

### Como utilizar la clase de ChatBot

chat.py es un script simple que se usa como ChatBot.

Hace algunas consultas y toma datos de usuario en tiempo real por teclado para interactuar con el modelo.

```python
import os
import logging

from libs.Chroma import ChromaManager
from libs.ChatBot import ChatBot
from config import ENV, CHROMADB_PATH, CSV_SOURCE, CSV_NER_PATH, HFACE_TOKEN


db = ChromaManager(use_local_fs=ENV=="colab", path=CHROMADB_PATH)

bot = ChatBot( db=db, name="Rick Sanchez", hface_token=HFACE_TOKEN)

intro = bot.welcome( lng="es")
print(intro)

esp_response = bot.ask(collection="cve", question="Generar una lista de vulnerabilidades en kernel de linux", k=10)
fr_response = bot.ask(collection="cve", question="Générer une liste de vulnérabilités dans le noyau Linux", k=10)
en_response = bot.ask(collection="cve", question="Make a list of linux kernel vulnerabilities", k=10)
esp_response = bot.ask(collection="cve", question="Necesito que me des informacion sobre esta vulnerabilidad <_ID_>CVE-2017-11302</_ID_/>", k=10)

for chat in bot.history:
    print("-------------------------------------------------")
    i = chat["idiom"]
    q = chat["question"]
    a = chat["response"]

    print(f"Idiom: {i}")
    print(f"Question: {q}")
    print(f"Answer: {a}")
    print("-------------------------------------------------")

while True:

    q = input("You: ")

    if q == "exit":
        break

    r = bot.ask(collection="cve", question=q, k=10)
    last_dialog = bot.history[-1]

    i = last_dialog["idiom"]
    q = last_dialog["question"]
    a = last_dialog["response"]

    print(f"Idiom Detected: {i}")
    print(f"Question Translated: {q}")
    print(f"Answer: {a}")
    print("-------------------------------------------------")
  ```


In [30]:
! export HFACE_TOKEN='TU_TOKEN' && export ENV='colab' && cd chatbot/src && python3 chat.py

Se truncaron las últimas líneas 5000 del resultado de transmisión.
-------------------------------------------------
Idiom: es
Question: generar una lista de vulnerabilidades en kernel de linux
Answer: 1. CVE-2021-31569: Existe una vulnerabilidad en el controlador de procesador Cortex-M (cortex_m) en el núcleo Linux que permite a un atacante local evacuar los recursos del sistema mediante la construcción de un dispositivo malicioso que e. 2. CVE-2022-0729: Una condición de carrera en el controlador ARM Trusted Execution Environment (ARM TEE) en el núcleo Linux puede ser explotada por un usuario malicioso local para obtener acceso privilegiado. 3. CVE-2022- 23176: Una vulnerabilidad de sobre flujo de buffer basada en la pila sale de la pila TCP/IP de la adelfia para Linux
-------------------------------------------------
-------------------------------------------------
Idiom: fr
Question: générer une liste de vulnérabilités dans le noyau linux
Answer: 1. CVE-2021-20716: Un problème a é

### Fine Tunning de un Modelo

Este script se utiliza como ejemplo para crear un nuevo modelo desde un checkpoint de GPT2.
Como es para modo academico y de estudio solo toma 10 registros de CVE y genera un nuevo modelo, pero si se tiene el suficiente poder de Hardware se podria utilizar el dataset entero y generar un modelo muy preciso.


```python
import logging, os
from libs.Trainers import ContextTransformerTrainer
from libs.datasets.BaseDataset import BaseDataset
from libs.datasets.Utils import cleaning

os.environ["TOKENIZERS_PARALLELISM"] = "true"

logging.basicConfig(level=logging.INFO)

model_output_path=f"./models/cve-model-gpt2-custom/"
model_checkpoint_path=f"microsoft/DialoGPT-medium"

trainer = ContextTransformerTrainer(
    model_output_dir=model_output_path,
    save_model_output_dir=model_output_path,
    special_tokens={"additional_special_tokens": ["<|cve|>"], 'pad_token': '[PAD]'},
    checkpoint=model_checkpoint_path,
    gpu=False
)

tokenizer = trainer.tokenizer
ds = BaseDataset(tokenizer)

if os.path.isfile("data/csv_train_gpt2111.csv"):
    path = "data/csv_train_gpt2.csv"
    save_df_to = None
    mapper = {}
else:
    path = "data/cve.csv"
    save_df_to = "data/csv_train_gpt2111.csv"
    mapper = {"feature": lambda x: "<|cve|>" + cleaning(x, max_length=512) + "<|endoftext|>"}

tokenized_data_batches = ds.load_csv_data(path=path, feature="Description",  truncate_before=0, truncate_after=10, mappers=mapper )
if save_df_to:
    ds.save(save_df_to)
    
ds.encode()
trainer.train(ds, resume_from_checkpoint=False)
trainer.save()
```

In [29]:
! cd chatbot/src && python3 train.py

2024-01-30 23:50:45.597948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 23:50:45.598011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 23:50:45.599927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 23:50:47.014129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal langua

# Ejercicio 2
---


## Introduccion
---


### Que es un Agente Inteligente ?
---


<p> Un agente inteligente es un programa de computadora o entidad autónoma diseñada para actuar de manera inteligente en un entorno específico con el fin de alcanzar objetivos o resolver problemas.</p> 
<p> Estos agentes pueden percibir su entorno a través de sensores, analizar la información recibida, tomar decisiones racionales basadas en esa información y luego actuar en su entorno para lograr sus metas o cumplir con sus objetivos. La inteligencia de estos agentes puede manifestarse en diferentes formas, como la capacidad de aprendizaje, la capacidad de razonamiento, la capacidad de planificación y la capacidad de adaptación a entornos cambiantes. 
<p>Los agentes inteligentes se utilizan en una variedad de aplicaciones, como sistemas de recomendación, asistentes virtuales, sistemas de control automático, robótica, juegos y sistemas de software complejos, entre otros.</p>
<p>En el panorama actual, observamos un desarrollo significativo en las aplicaciones de agentes inteligentes basados en modelos LLM libres.
Estos agentes han experimentado un progreso notable en diversas tareas relacionadas con el procesamiento NLP y Machine Learning.</p>

<p>Estos agentes están transformando la interacción con el lenguaje natural y abriendo nuevas posibilidades en una variedad de tareas. A medida que la tecnología avanza, se anticipa que estos agentes se volverán aún más sofisticados, abordando desafíos más complejos y facilitando la resolución de problemas en diversos campos de aplicación. </p>

## Tipos y Caracteristicas
---

### Aplicaciones en Procesamiento del Lenguaje Natural (NLP)
---

* <u>Traducción Automática</u> :
Los agentes inteligentes basados en LLM libres han alcanzado logros que se destecan en la esfera de la traducción automática. Modelos como MADLAD400 y mBART han contribuido significativamente a mejorar la precisión y fluidez en la traducción entre diferentes idiomas.

* <u>Resumen de Texto</u> :
El agente Zephyr-7B destaca en la generación de resúmenes, produciendo textos más concisos sin comprometer la esencia informativa. Esta capacidad facilita la comprensión y extracción eficiente de la información relevante.

* <u> Detección de Emociones</u> :
Modelos como RoBERTa-spam han demostrado un notable aumento en la precisión al identificar matices emocionales en el texto. Esta habilidad tiene aplicaciones importantes en el análisis de sentimientos y la comprensión contextual de mensajes en entornos virtuales.

* <u> Generación de Texto Creativo </u> :
El agente Llama-2 ha elevado la generación de texto creativo a niveles excepcionales, abarcando desde la producción de poemas hasta código y composiciones musicales de manera coherente y expresiva.

### Aplicaciones en Aprendizaje Automático (ML)
---

* <u>Clasificación de Texto</u> :
Los modelos LLM libres, como notdiamond-0001, han demostrado mejoras sustanciales en la precisión de la clasificación de texto en diversos contextos, desde noticias hasta comentarios en redes sociales, permitiendo una adaptabilidad significativa a diferentes dominios.

* <u>Regresión</u>:
En el ámbito de la regresión, modelos como Opus han exhibido eficacia en la predicción de valores continuos en textos. Esta capacidad resulta esencial para estimar variables como la longitud de un texto o la puntuación asociada a su contenido.

* <u> Detección de Anomalías </u>
Agentes como RoBERTa-spam han sido empleados exitosamente para la detección de anomalías en textos, como errores ortográficos, contenido ofensivo y posibles fraudes.

* <u> Tendencias Emergentes y Perspectivas Futuras </u>
El desarrollo de agentes inteligentes basados en LLM libres sigue una tendencia hacia la especialización y personalización de aplicaciones. Además, la integración de técnicas de aprendizaje por refuerzo y la mejora en la interpretabilidad de los modelos son áreas activas de investigación que perfilan el futuro de estos agentes.

## Ejemplo de problematica a solucionar con Sistemas MultiAgentes
---

**Problemática: Detección y Respuesta a Amenazas en un Sistema de Seguridad Informática**
---

**¿Cuáles son los beneficios de un entorno de ciberseguridad mejorado por LLM?**
<p>Los modelos de lenguaje grandes no son solo para escritura creativa; también tienen un papel crucial en ciberseguridad. </p>
<p>En un mundo donde las amenazas cibernéticas están siempre presentes, estos modelos pueden ayudar en la detección avanzada de amenazas, analizando datos de texto para identificar vulnerabilidades y mejorando la respuesta a incidentes a través del contexto.</p> 
<p>El PLN puede ahora entender tanto el lenguaje humano como los encabezados generados por máquinas, lo que ayuda a identificar indicadores de intentos de phishing o ataques a plataformas expuestas a internet mediante analisis de paquetes de red, entre otras cosas. </p>
<p>En entornos empresariales, la seguridad informática es crucial para proteger la integridad y la confidencialidad de la información. La problemática a abordar es la detección y respuesta eficiente a amenazas de cyber seguridad en tiempo real. Se propone un sistema multiagente para mejorar la seguridad informática mediante la colaboración de diversos agentes especializados.</p>

**Agentes Involucrados:**

1. **Agente de Detección de Amenazas:**

   - [Fuente](https://arxiv.org/abs/2306.14263) / [Paper](https://arxiv.org/pdf/2306.14263.pdf)
   - Modelo : SecurityBERT ( finetunning basado en BERT )

   <p>Para resumir el extenso paper de SecurityBERT el modelo filtra los paquetes de red, filtrando varias capas de los protocolos y tokenizando los datos para poder hacer fine tuning sobre BERT.</p>

   El agente basado en este modelo tendra las siguientes caracteristicas :
   - Encargado de monitorear continuamente la red en busca de patrones y comportamientos sospechosos.
   - Utiliza algoritmos de aprendizaje automático para identificar posibles amenazas.
   - Genera contexto con la IP de la fuente y destino, y la aplicacion de destino del trafico malicioso detectado.
   - Genera un ID para el incidente para poder responder y propociarnas mas detalles en el futuro si se necesita.
   - Ejemplo de Conversación:
     - ADA: "He detectado un patrón inusual de tráfico en la red, posiblemente una actividad maliciosa, hacia la aplicacion <b>ExampleAPP</b>. El ID del incidente: INCIDENTE_ID"
     - Agente de Respuesta a Incidentes: "Entendido, actuaré de inmediato."

   - ADA generara un evento en la Queue de mensajes( Kafka/PubSub/SQS ) con el ID del incidente donde los otros agentes estan suscriptos y podran acceder a los metadatos y la informacion que necesiten.

2. **Agente de Información de Vulnerabilidades (AIV):**
   - Modelo: Fine tuned model basado en Llama o GPT para clasificar vulnerabilidades de las librerias y paquetes de los sitemas operativos utilizados en el sistema. El modelo sera entrenado como explica las fuentes citadas para clasificar las distintas vulnerabilidades en baja, media o critica.
   - Fuentes :
     - [Fine-tuning Llama 2 for news category prediction: A step-by-step comprehensive guide to fine-tuning any LLM ](https://medium.com/@kshitiz.sahay26/fine-tuning-llama-2-for-news-category-prediction-a-step-by-step-comprehensive-guide-to-fine-tuning-48c06dee28a9)
     - [Large Language Models for Text Classification](https://osf.io/sthwk/download/?format=pdf)

   - Este agente es el encargado de recopilar información sobre nuevas vulnerabilidades y amenazas emergentes.
   - Ejemplo de Conversación:
     - AIV: "Vulnerabilidad CRITICA detectada para las libreria ExpressJS de nodejs en su version <=2.0.1. CVE reportado CV-ID-EXAMPLE."
      

3. **Agente de Respuesta a Incidentes:**
   - Modelo : Fine tuned BERT model utilizando Headers y paquetes TCP/IP para el entrenamiento.
   - Fuente : [(NER) Model Fine-Tuning with LLM](https://ugurozker.medium.com/turkish-named-entity-recognition-ner-model-fine-tuning-with-llm-09654c054ae5). 

   <p>Si bien este articulo explica como utilizar fine tunning de un modelo de BERT para el idioma turco, podria aplicarse para nuestro caso practico, entrenando el modelo para detectar entidades en versiones de parquetes o librerias y poder actuar automaticamente para el patcheo de vulnerabilidades.
   Tambien detectar entidades en paquetes de RED como Host, Agents, y otros Headers que puedan servir para actualizar automaticamente reglas de Firewall o WAF.</p>

   <p>El agente que utiliza este modelo, tendra la siguientes caracteristicas : </p>

   - Responsable de analizar y responder a posibles amenazas detectadas por ADA.
   - Coordina con otros agentes de seguridad para tomar medidas correctivas.
   - Ejemplo de Conversación:
     - ARI: "ADA ha identificado un posible ataque de malware. Voy a aislar la máquina afectada y actualizar las reglas de WAF para bloquear el trafico proveniente de la IP X.X.X.X , utilizando Agent: NombreAgente con posible ataques de SQL Injection hacia <b> </b>."
     - ARI: "AIV detecto una nueva vulnerabilidad clasificada como critica para la libreria express. Voy a escanear los repositorios de las aplicaciones para ejecutar build y deploy para detectar y actulizar las aplicaciones afectadas."


<p>Este sistema multiagente permite una respuesta rápida y coordinada ante amenazas cibernéticas, mejorando la seguridad informática mediante la detección temprana, la respuesta eficiente y la implementación proactiva de medidas preventivas.</p>

**Ejemplos**
Existen empresas que estan innovando en integrar agentes basados en modelos LLM para automatizar cuestiones de seguridad en las empresas. Un ejemplo de estos agentes es el reciente [Torq Socrates](https://venturebeat.com/security/torq-launches-torq-socrates-an-ai-agent-for-tier-1-secops-threat-resolution/).

**Mas Fuentes**: 
- SecurityBERT : [Fuente](https://arxiv.org/abs/2306.14263) / [Paper](https://arxiv.org/pdf/2306.14263.pdf)
- [Richardson B. (2022) Changing Cybersecurity with Natural Language Processing](https://developer.nvidia.com/blog/changing-cybersecurity-with-natural-language-processing/)
- [Peacock J. How NLP is Transforming Cyber Risk and Compliance.](https://www.cybersaint.io/blog/ai-cybersecurity)
- [Large Language Models in Cybersecurity.](https://gradientflow.com/large-language-models-in-cybersecurity/)
- [Fine-tuning Llama 2 for news category prediction: A step-by-step comprehensive guide to fine-tuning any LLM ](https://medium.com/@kshitiz.sahay26/fine-tuning-llama-2-for-news-category-prediction-a-step-by-step-comprehensive-guide-to-fine-tuning-48c06dee28a9)
- [Iwugo Daniel. Large Language Models and Cybersecurity](https://www.freecodecamp.org/news/large-language-models-and-cybersecurity/)
- [Large Language Models for Text Classification](https://osf.io/sthwk/download/?format=pdf)

### Diagrama de Arquitectura
---

<img src="http://drive.google.com/uc?export=view&id=1Yxzuz_-rAkrhgzFm6SY4O1Cw_Z-D_GmG">